<a href="https://colab.research.google.com/github/AlenaAntipina/KungfuGameDetector/blob/main/detector_kungfu_link_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12313, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 12313 (delta 14), reused 6 (delta 2), pack-reused 12281
Receiving objects: 100% (12313/12313), 12.08 MiB | 29.51 MiB/s, done.
Resolving deltas: 100% (8489/8489), done.


In [ ]:
!pip install -r yolov5/requirements.txt

In [ ]:
with open('new_train_yaml', 'w+') as file:
    file.write(
        """
        # parameters
        nc: 3  # number of classes
        depth_multiple: 0.33  # model depth multiple
        width_multiple: 0.50  # layer channel multiple

        # anchors
        anchors:
          - [10,13, 16,30, 33,23]  # P3/8
          - [30,61, 62,45, 59,119]  # P4/16
          - [116,90, 156,198, 373,326]  # P5/32

        # YOLOv5 backbone
        backbone:
          # [from, number, module, args]
          [[-1, 1, Focus, [64, 3]],  # 0-P1/2
           [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
           [-1, 3, BottleneckCSP, [128]],
           [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
           [-1, 9, BottleneckCSP, [256]],
           [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
           [-1, 9, BottleneckCSP, [512]],
           [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
           [-1, 1, SPP, [1024, [5, 9, 13]]],
           [-1, 3, BottleneckCSP, [1024, False]],  # 9
          ]

        # YOLOv5 head
        head:
          [[-1, 1, Conv, [512, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 6], 1, Concat, [1]],  # cat backbone P4
           [-1, 3, BottleneckCSP, [512, False]],  # 13

           [-1, 1, Conv, [256, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 4], 1, Concat, [1]],  # cat backbone P3
           [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

           [-1, 1, Conv, [256, 3, 2]],
           [[-1, 14], 1, Concat, [1]],  # cat head P4
           [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

           [-1, 1, Conv, [512, 3, 2]],
           [[-1, 10], 1, Concat, [1]],  # cat head P5
           [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

           [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
          ]
        """
    )

In [ ]:
with open('new_data_yaml', 'w+') as file:
    file.write(
        """
        train: /content/gdrive/MyDrive/Detector/dataset_game/images/train
        val: /content/gdrive/MyDrive/Detector/dataset_game/images/valid

        nc: 3
        names: ['NPC', 'Hero', 'bullet']
        """
    )

In [ ]:
!python /content/yolov5/train.py --img 416 --batch 16 --epochs 500 --data /content/new_data_yaml --cfg /content/new_train_yaml

   309/499     1.89G  0.007246  0.003009 0.0003529        15       416: 100% 100/100 [00:21<00:00,  4.56it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 10/10 [00:02<00:00,  4.94it/s]
                 all        301        410      0.999      0.996      0.995      0.955

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   310/499     1.89G   0.00782  0.003141 0.0004635        18       416: 100% 100/100 [00:21<00:00,  4.57it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 10/10 [00:02<00:00,  4.84it/s]
                 all        301        410      0.999      0.996      0.995      0.955

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   311/499     1.89G  0.007168  0.002957  0.000371        13       416: 100% 100/100 [00:22<00:00,  4.52it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 10/10 [00:0

In [ ]:
# !python /content/yolov5/train.py --img 416 --batch 16 --epochs 200 --data /content/new_data_yaml --cfg /content/new_train_yaml --weights /content/yolov5/runs/train/exp/weights/last.pt

In [ ]:
!python /content/yolov5/detect.py --source /content/gdrive/MyDrive/Detector/image3.png --weights '/content/yolov5/runs/train/exp/weights/best.pt' --img 416 --conf 0.5 --save-txt

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/gdrive/MyDrive/Detector/image3.png, data=yolov5/data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-265-g8ebf569 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
new_train_yaml summary: 232 layers, 7251912 parameters, 0 gradients
image 1/1 /content/gdrive/MyDrive/Detector/image3.png: 224x416 1 bullet, Done. (0.017s)
Speed: 0.4ms pre-process, 16.9ms inference, 1.3ms NMS per image at shape (1, 3, 416, 416)
Results saved to yolov5/runs/detect/exp4
1 labels saved to yolov5/runs/detect/exp4/labels


In [ ]:
import torch
import cv2
from PIL import Image
import pandas

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/runs/train/exp/weights/best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-6-27 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
new_train_yaml summary: 232 layers, 7251912 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
from PIL import Image
from numpy import asarray

# load the image and convert into numpy array
img = Image.open('Sample.png')

# asarray() class is used to convert PIL images into NumPy arrays
numpydata = asarray(img)

# <class 'numpy.ndarray'>
print(type(numpydata))

#  shape
print(numpydata.shape)

In [ ]:
image = '/content/gdrive/MyDrive/Detector/image2.png'
results = model(image)
results.pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,56.712124,31.855856,68.648643,62.362316,0.960145,1,bullet
1,49.642105,25.497768,56.354744,58.142731,0.954689,0,Hero


In [ ]:
results.print()

image 1/1: 64x128 1 Hero, 1 bullet
Speed: 14.2ms pre-process, 57.1ms inference, 1.6ms NMS per image at shape (1, 3, 320, 640)
